In [1]:
seed = 2020

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext import data
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
from torchtext.vocab import Vectors
import time
import heapq
import math
import random
from tqdm import tqdm
from torchtext.vocab import GloVe
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from nltk import word_tokenize

In [3]:
import pandas as pd

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(0)

In [5]:
tokenize = word_tokenize
Article = data.Field(sequential=True, tokenize=tokenize, lower=True, init_token='<bos>', eos_token='<eos>',
                  pad_token='<pad>', unk_token='<oov>')
Title = data.Field(sequential=True, tokenize=tokenize, lower=True, eos_token='<eos>', 
                  pad_token='<pad>', unk_token='<oov>')
Len = data.Field(sequential=False, use_vocab=False)
ID = data.Field(sequential=False, use_vocab=False)

In [6]:
pmi_dct = dict()

In [7]:
train_path = 'train_data.csv'
valid_path = 'valid_data.csv'
test_path = 'test_data.csv'


class MyDataset(data.Dataset):
    def __init__(self, csv_path, article_field, title_field, cat, **kwargs):

        csv_data = pd.read_csv(csv_path)
        fields = [("id", ID), ("article", article_field), ("title", title_field), ("art_len", Len), ("title_len", Len)]
        examples = []
    
        
        for id, text, label in tqdm(zip(csv_data.index, csv_data['article'], csv_data['title'])):
            examples.append(data.Example.fromlist([self.getID(id, cat), text, label, len(word_tokenize(text))+2, len(word_tokenize(label))+1], fields))
        super(MyDataset, self).__init__(examples, fields)

    def getID(self, id, cat):
        if cat == 'train':
            return id
        elif cat == 'valid':
            return id + 100000
        elif cat == 'test':
            return id + 110000
        
    def shuffle(self, text):
        text = np.random.permutation(text.strip().split())
        return ' '.join(text)


# class MyDataset(data.Dataset):
#     def __init__(self, csv_data, article_field, title_field, cat, **kwargs):
#         if cat == 'train':
#             csv_data = csv_data[:200000]
#         elif cat == 'valid':
#             csv_data = csv_data[200000:220000]
#         elif cat == 'test':
#             csv_data = csv_data[220000:]
        
#         fields = [("id", ID), ("article", article_field), ("title", title_field), ("art_len", Len), ("title_len", Len)]
#         examples = []
    
        
#         for id, text, label in tqdm(zip(csv_data.index, csv_data['article'], csv_data['title'])):
#             examples.append(data.Example.fromlist([id, text, label, len(word_tokenize(text))+2, len(word_tokenize(label))+1], fields))
#         super(MyDataset, self).__init__(examples, fields)

        
        
#     def shuffle(self, text):
#         text = np.random.permutation(text.strip().split())
#         return ' '.join(text)

In [8]:
train = MyDataset(train_path, article_field=Article, title_field=Title, cat='train')
valid = MyDataset(valid_path, article_field=Article, title_field=Title, cat='valid')
test = MyDataset(test_path, article_field=Article, title_field=Title, cat='test')

100000it [01:29, 1112.46it/s]
10000it [00:08, 1131.51it/s]
10000it [00:09, 1082.34it/s]


In [9]:
from collections import defaultdict
vectors = Vectors(name='glove-wiki-gigaword-300.txt')
Article.build_vocab(train, valid, test, vectors=vectors)
default = Article.vocab.stoi['<oov>']
Article.vocab.stoi.default_factory = lambda: default
Title.vocab = Article.vocab

In [10]:
from torchtext.data import Iterator, BucketIterator
train_iter, val_iter = BucketIterator.splits(
        (train, valid), 
        batch_size=16, 
        device=device, 
        sort_key=lambda x: len(x.article),
        sort_within_batch=True,
        repeat=False 
)


## GCN

In [11]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, device, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features)).to(device)
        self.device = device
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features)).to(device)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import math
import numpy as np


class GCN(nn.Module):
    def __init__(self, input_dim, nfeat, nhid, nout, dropout, device):
        super(GCN, self).__init__()
        self.input_dim = input_dim
        self.nfeat = nfeat
        self.gc1 = GraphConvolution(nfeat, nhid, device)
        self.gc2 = GraphConvolution(nhid, nout, device)
        self.device = device
        self.dropout = dropout

    def forward(self, input_seq, seq_embed, input_length, id=None):
        pad_len = len(input_seq)
        input_seq = input_seq[:input_length]
        adj, uni_words = self.load_data(input_seq, id)
        uni2idx = [list(input_seq).index(ele) for ele in uni_words]
        seq2idx = [list(uni_words).index(ele) for ele in input_seq]
        x = seq_embed[uni2idx]
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        x = torch.tanh(x) # [node_num, embded_dim]
        x = x[seq2idx]  # [seq_len, embed_dim]
        if pad_len > input_length:
            diff = pad_len - input_length
            pad_embed = torch.tensor(seq_embed[-1,:].unsqueeze(0).expand(diff, self.nfeat), device=self.device)
            x = torch.cat((x,pad_embed),0)   # [pad_len, emb_dim]
        return x
        
    def unranked_unique(self, nparray):
        n_unique = len(np.unique(nparray))
        ranked_unique = np.zeros([n_unique])
        i = 0
        for x in nparray:
            if x not in ranked_unique:
                ranked_unique[i] = x
                i += 1
        return ranked_unique

    def load_data(self, word_seq, id):
        if id is not None:
            id = int(id)
        word_seq = torch.tensor(word_seq, device=self.device)
        words = torch.unique(word_seq, sorted=False)
        word_num = len(words)
        if id is None:
            adj = self.get_pmi(word_seq, words, 3, self.device)
        elif id in pmi_dct.keys():
            adj = pmi_dct[id]
        else:
            adj = self.get_pmi(word_seq, words, 3, self.device)
            adj = torch.tensor(self.normalize(adj), dtype=torch.float32, device=self.device)
            pmi_dct[id] = adj
        words = torch.tensor(words, dtype=torch.int64, device=self.device)
        return adj, words

    def get_pmi(self, word_seq, uni_words, window_size, device):
        word_len = len(word_seq)
        word_num = len(uni_words)
        seq2idx = {int(ele): list(uni_words).index(ele) for ele in uni_words}
        window_num = word_len - window_size + 1
        if window_size > word_len:
            window_size = word_len
        win_num_matrix = torch.zeros((word_num, word_num), device=device)

        for i in range(window_num):
            words = list(torch.unique(word_seq[i:i+window_size], sorted=False))
            indicies = [seq2idx[int(ele)] for ele in words]
            for ele in indicies:
                win_num_matrix[ele,indicies] += 1

        pos_matrix = win_num_matrix / window_num
        diag = torch.mul(pos_matrix, torch.eye(word_num, device=device)).inverse()
        pos_matrix = torch.mm(diag, pos_matrix)
        pos_matrix = torch.mm(pos_matrix, diag)
        pmi_matrix = torch.clamp(torch.log2(pos_matrix), min=0)
        diag = torch.diag(pmi_matrix)
        diag = 1 - diag
        pmi_matrix = pmi_matrix + torch.diag_embed(diag)
        return pmi_matrix

    def normalize(self, mx):
        """Symmetrically normalize adjacency matrix"""
        rowsum = mx.sum(1)
        r_inv = torch.pow(rowsum, -0.5).flatten()
        r_inv = torch.clamp(r_inv, min=0)
        r_mat_inv = torch.diag(r_inv)
        mx = torch.mm(r_mat_inv, mx)
        mx = torch.mm(mx, r_mat_inv)
        return mx


## GAT

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nout, dropout, alpha, nheads, device):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout
        self.device = device
        self.nfeat = nfeat
        self.nhid = nhid
        self.nout = nout
        self.nheads = nheads
        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, device=device, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GraphAttentionLayer(nhid * nheads, nout, dropout=dropout, alpha=alpha, device=device, concat=False)

#     def forward(self, x, adj):
#         x = F.dropout(x, self.dropout, training=self.training)
#         x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
#         x = F.dropout(x, self.dropout, training=self.training)
#         x = F.elu(self.out_att(x, adj))
#         return F.log_softmax(x, dim=1)

    def forward(self, input_seq, seq_embed, input_length, id=None):
        pad_len = len(input_seq)
        input_seq = input_seq[:input_length]
        adj, uni_words = self.load_data(input_seq, id)
        uni2idx = [list(input_seq).index(ele) for ele in uni_words]
        seq2idx = [list(uni_words).index(ele) for ele in input_seq]
        x = seq_embed[uni2idx]
        # x: [node_num, feature_dim]
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.elu(self.out_att(x, adj))
        x = torch.tanh(x) # [node_num, embded_dim]
        x = x[seq2idx]  # [seq_len, embed_dim]
        if pad_len > input_length:
            diff = pad_len - input_length
            pad_embed = torch.tensor(seq_embed[-1,:].unsqueeze(0).expand(diff, self.nfeat), device=self.device)
            x = torch.cat((x,pad_embed),0)   # [pad_len, emb_dim]
        return x

        
    def unranked_unique(self, nparray):
        n_unique = len(np.unique(nparray))
        ranked_unique = np.zeros([n_unique])
        i = 0
        for x in nparray:
            if x not in ranked_unique:
                ranked_unique[i] = x
                i += 1
        return ranked_unique

    def load_data(self, word_seq, id):
        if id is not None:
            id = int(id)
        word_seq = torch.tensor(word_seq, device=self.device)
        words = torch.unique(word_seq, sorted=False)
        word_num = len(words)
        if id is None:
            adj = self.get_pmi(word_seq, words, 3, self.device)
        elif id in pmi_dct.keys():
            adj = pmi_dct[id]
        else:
            adj = self.get_pmi(word_seq, words, 3, self.device)
            adj = torch.tensor(self.normalize(adj), dtype=torch.float32, device=self.device)
            pmi_dct[id] = adj
        words = torch.tensor(words, dtype=torch.int64, device=self.device)
        return adj, words

    def get_pmi(self, word_seq, uni_words, window_size, device):
        word_len = len(word_seq)
        word_num = len(uni_words)
        seq2idx = {int(ele): list(uni_words).index(ele) for ele in uni_words}
        window_num = word_len - window_size + 1
        if window_size > word_len:
            window_size = word_len
        win_num_matrix = torch.zeros((word_num, word_num), device=device)

        for i in range(window_num):
            words = list(torch.unique(word_seq[i:i+window_size], sorted=False))
            indicies = [seq2idx[int(ele)] for ele in words]
            for ele in indicies:
                win_num_matrix[ele,indicies] += 1

        pos_matrix = win_num_matrix / window_num
        diag = torch.mul(pos_matrix, torch.eye(word_num, device=device)).inverse()
        pos_matrix = torch.mm(diag, pos_matrix)
        pos_matrix = torch.mm(pos_matrix, diag)
        pmi_matrix = torch.clamp(torch.log2(pos_matrix), min=0)
        diag = torch.diag(pmi_matrix)
        diag = 1 - diag
        pmi_matrix = pmi_matrix + torch.diag_embed(diag)
        return pmi_matrix

    def normalize(self, mx):
        """Symmetrically normalize adjacency matrix"""
        rowsum = mx.sum(1)
        r_inv = torch.pow(rowsum, -0.5).flatten()
        r_inv = torch.clamp(r_inv, min=0)
        r_mat_inv = torch.diag(r_inv)
        mx = torch.mm(r_mat_inv, mx)
        mx = torch.mm(mx, r_mat_inv)
        return mx


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """
    def __init__(self, in_features, out_features, dropout, alpha, device, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.device =device
        self.out_features = out_features
        self.alpha = alpha         # 学习因子
        self.concat = concat
        # 建立就是0的矩阵，大小为(输入维度，输出维度)
        self.W = nn.Parameter(torch.empty(size=(in_features, out_features))).to(device)
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        # self.a.shape [16, 1], 因为GAT的注意力机制打分用的是拼接方法，所以这里的a维度是[2*out_features, 1]
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1))).to(device)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        # adj: 邻接矩阵， 维度： N*N
        # h: [N, in_features]
        Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
        a_input = self._prepare_attentional_mechanism_input(Wh)  # a_input: [N, N, 2*out_features]
        # self.a: [2*out_features, 1]
        # e: [N, N]
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
        # 这里采用了Local Attention 机制，即只对每个节点的一节邻节点进行聚合。所以对于一个邻接矩阵，只对值为1的位置赋值。
        zero_vec = -9e15*torch.ones_like(e)
        # attention 是一个N*N的矩阵，
#         print(adj.shape, e.shape, zero_vec.shape)
        
        attention = torch.where(adj > 0, adj, zero_vec)

        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        # Wh: [N, out_features]

        N = Wh.size()[0] # number of nodes

        # Below, two matrices are created that contain embeddings in their rows in different orders.
        # (e stands for embedding)
        # These are the rows of the first matrix (Wh_repeated_in_chunks): 
        # e1, e1, ..., e1,            e2, e2, ..., e2,            ..., eN, eN, ..., eN
        # '-------------' -> N times  '-------------' -> N times       '-------------' -> N times
        # 
        # These are the rows of the second matrix (Wh_repeated_alternating): 
        # e1, e2, ..., eN, e1, e2, ..., eN, ..., e1, e2, ..., eN 
        # '----------------------------------------------------' -> N times
        # 
        
        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=0)  # Wh_repeated_in_chunks: []
        Wh_repeated_alternating = Wh.repeat(N, 1)
        # Wh_repeated_in_chunks.shape == Wh_repeated_alternating.shape == (N * N, out_features)

        # The all_combination_matrix, created below, will look like this (|| denotes concatenation):
        # e1 || e1
        # e1 || e2
        # e1 || e3
        # ...
        # e1 || eN
        # e2 || e1
        # e2 || e2
        # e2 || e3
        # ...
        # e2 || eN
        # ...
        # eN || e1
        # eN || e2
        # eN || e3
        # ...
        # eN || eN

        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
        # all_combinations_matrix.shape == (N * N, 2 * out_features)

        return all_combinations_matrix.view(N, N, 2 * self.out_features)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [15]:
class Encoder(nn.Module):
    def __init__(self, gnn, input_dim, emb_dim, hid_dim, n_layers, device, dropout=0.5, bidirectional=True):
        super(Encoder, self).__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.GNN = gnn
        self.embedding = nn.Embedding(input_dim, emb_dim)
        weight_matrix = Article.vocab.vectors
        self.embedding.weight.data.copy_(weight_matrix)
        self.device = device
        self.gru = nn.GRU(emb_dim*2, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
    def forward(self, input_seqs, input_lengths, ids, hidden):
        # input_seqs = [seq_len, batch]
        embedded = self.embedding(input_seqs)
        # embedded = [pad_len, batch, embed_dim]
        gnn_outputs = torch.zeros(embedded.shape, device=self.device)
        for i in range(embedded.shape[1]):
            gnn_input = embedded[:,i,:]
            if ids is not None:
                gnn_output = self.GNN(input_seqs[:,i], gnn_input, input_lengths[i], ids[i]) # pad_len, emb_dim
            else:
                gnn_output = self.GNN(input_seqs[:,i], gnn_input, input_lengths[i])
            gnn_outputs[:,i,:] = gnn_output
        inputs = torch.cat((embedded, gnn_outputs),-1)
        packed = torch.nn.utils.rnn.pack_padded_sequence(inputs, input_lengths, enforce_sorted=False)
        
        outputs, hidden = self.gru(packed, hidden)        
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # outputs = [seq_len, batch, hid_dim * n directions]
        # output_lengths = [batch]
        return outputs, hidden


In [16]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout=0.5, bidirectional=True):
        super(Decoder, self).__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        weight_matrix = Article.vocab.vectors
        self.embedding.weight.data.copy_(weight_matrix)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
        if bidirectional:
            self.fc_out = nn.Linear(hid_dim*2, output_dim)
        else:
            self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, token_inputs, hidden):
        # token_inputs = [batch]
        batch_size = token_inputs.size(0)
        embedded = self.dropout(self.embedding(token_inputs).view(1, batch_size, -1))
        # embedded = [1, batch, emb_dim]
        output, hidden = self.gru(embedded, hidden)
        # output = [1, batch,  n_directions * hid_dim]
        # hidden = [n_layers * n_directions, batch, hid_dim]
        
        output = self.fc_out(output.squeeze(0))
        output = self.softmax(output)
        # output = [batch, output_dim]
        return output, hidden

In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 device, 
                 predict=False, 
                 basic_dict=None,
                 max_len=100
                 ):
        super(Seq2Seq, self).__init__()
        
        self.device = device

        self.encoder = encoder
        self.decoder = decoder

        self.predict = predict  
        self.basic_dict = basic_dict  
        self.max_len = max_len  

        self.enc_n_layers = self.encoder.gru.num_layers
        self.enc_n_directions = 2 if self.encoder.gru.bidirectional else 1
        self.dec_n_directions = 2 if self.decoder.gru.bidirectional else 1

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        assert self.enc_n_directions >= self.dec_n_directions, \
            "If decoder is bidirectional, encoder must be bidirectional either!"
        
    def forward(self, input_batches, input_lengths, target_batches=None, target_lengths=None, ids=None, teacher_forcing_ratio=0.5):
        # input_batches = target_batches = [seq_len, batch]
        batch_size = input_batches.size(1)
        BOS_token = self.basic_dict["<bos>"]
        EOS_token = self.basic_dict["<eos>"]
        PAD_token = self.basic_dict["<pad>"]

        encoder_hidden = torch.zeros(self.enc_n_layers*self.enc_n_directions, batch_size, self.encoder.hid_dim, device=self.device)
        
        # encoder_output = [seq_len, batch, hid_dim * n directions]
        # encoder_hidden = [n_layers*n_directions, batch, hid_dim]
        encoder_output, encoder_hidden = self.encoder(
            input_batches, input_lengths, ids, encoder_hidden)

        decoder_input = torch.tensor([BOS_token] * batch_size, dtype=torch.long, device=self.device)
        if self.enc_n_directions == self.dec_n_directions:
            decoder_hidden = encoder_hidden
        else:
            L = encoder_hidden.size(0)
            decoder_hidden = encoder_hidden[range(0, L, 2)] + encoder_hidden[range(1, L, 2)]
            
        if self.predict:
            assert batch_size == 1, "batch_size of predict phase must be 1!"
            output_tokens = []

#             return beam_search(3, decoder_inpuy, decoder_hidden)
            prev_beam = Beam()
            prev_beam.add(0, False, [decoder_input], decoder_input, decoder_hidden)
            while True:
                cur_beam = Beam()
                for prob, complete, seq_list, decoder_input, decoder_hidden in prev_beam:
                    if complete:
                        cur_beam.add(prob, complete, seq_list, decoder_input, decoder_hidden)
                    else:
                        # decoder_output_t: [batch_size, vocab_dim]
                        # decoder_hidden: [num_layer*num_direction, batch_size, hidden_dim]
                        decoder_output_t, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                        # value: [batch_size, beam_width] log_softmax, 
                        # index: [batch_size, beam_width]
                        value, index = torch.topk(decoder_output_t, 3)
                        seq_len = 0
                        for m, n in zip(value[0], index[0]):
                            seq_len += 1
                            cur_prob = (prob+m.item())/ seq_len
                            decoder_input = torch.LongTensor([n.item()]).to(self.device)
                            cur_seq_list = seq_list + [decoder_input]
                            if int(n) == EOS_token or len(cur_seq_list) == self.max_len:
                                cur_complete = True
                            else:
                                cur_complete = False
                            cur_beam.add(cur_prob, cur_complete, cur_seq_list, decoder_input, decoder_hidden)
                    
                best_prob, best_complete, best_seq, _, _ = max(cur_beam)
                if best_complete or len(best_seq) >= self.max_len: 
                    best_seq = [i.item() for i in best_seq]
                    return best_seq
                else:
                    prev_beam = cur_beam
#         if self.predict:
#             assert batch_size == 1, "batch_size of predict phase must be 1!"
#             output_tokens = []

#             while True:
#                 decoder_output, decoder_hidden = self.decoder(
#                     decoder_input, decoder_hidden
#                 )
#                 # [1, 1]
#                 topv, topi = decoder_output.topk(1)
#                 decoder_input = topi.squeeze(1)  
#                 output_token = topi.squeeze().detach().item()
#                 if output_token == EOS_token or len(output_tokens) == self.max_len:
#                     break
#                 output_tokens.append(output_token)
#             return output_tokens

        else:
            max_target_length = max(target_lengths)
            all_decoder_outputs = torch.zeros((max_target_length, batch_size, self.decoder.output_dim), device=self.device)

            for t in range(max_target_length):
                use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
                if use_teacher_forcing:
                    # decoder_output = [batch, output_dim]
                    # decoder_hidden = [n_layers*n_directions, batch, hid_dim]
                    decoder_output, decoder_hidden = self.decoder(
                        decoder_input, decoder_hidden
                    )
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = target_batches[t]  
                else:
                    decoder_output, decoder_hidden = self.decoder(
                        decoder_input, decoder_hidden
                    )
                    # [batch, 1]
                    topv, topi = decoder_output.topk(1)
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = topi.squeeze(1)  
     
            loss_fn = nn.NLLLoss(ignore_index=PAD_token)
            loss = loss_fn(
                all_decoder_outputs.reshape(-1,self.decoder.output_dim ),  # [batch*seq_len, output_dim]
                target_batches.reshape(-1)                                                 # [batch*seq_len]
            )
            return loss

### Beam Search

In [18]:
class Beam:
    """保存每一个时间步的数据"""
 
    def __init__(self, beam_width=1):
        self.heapq = list()
        self.beam_width = beam_width
 
    def add(self, prob, complete, seq_list, decoder_input, decoder_hidden):
        heapq.heappush(self.heapq, [prob, complete, seq_list, decoder_input, decoder_hidden])
        # 保证最终只有一个beam width个结果
        if len(self.heapq) > self.beam_width:
            heapq.heappop(self.heapq)
 
    def __iter__(self):
        for item in self.heapq:
            yield item

In [19]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [20]:
def train(
    model,
    data_loader, 
    optimizer, 
    clip=1, 
    teacher_forcing_ratio=0.5, 
    print_every=1 
    ):
    model.predict = False
    model.train()

    if print_every == 0:
        print_every = 1

    print_loss_total = 0  
    start = time.time()
    epoch_loss = 0
    step = 0
    for batch in tqdm(data_loader, position=0, leave=True):
        step += 1
        ids = batch.id
        input_batchs = batch.article
        target_batchs = batch.title
        input_lens = list(batch.art_len)
        target_lens = list(batch.title_len)
        optimizer.zero_grad()
        loss = model(input_batchs, input_lens, target_batchs, target_lens, ids, teacher_forcing_ratio)
        print_loss_total += loss.item()
        epoch_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        if step % 500 == 0:
            print_loss_avg = print_loss_total / 500
            print_loss_total = 0
            print('\tCurrent Loss: %.4f' % print_loss_avg)

    return epoch_loss / len(data_loader)

In [21]:
def evaluate(
    model,
    data_loader, 
    print_every=None
    ):
    model.predict = False
    model.eval()
    if print_every == 0:
        print_every = 1

    print_loss_total = 0  
    start = time.time()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            ids = batch.id
            input_batchs = batch.article
            target_batchs = batch.title
            input_lens = list(batch.art_len)
            target_lens = list(batch.title_len)
        

            loss = model(input_batchs, input_lens, target_batchs, target_lens, ids, teacher_forcing_ratio=0)
            print_loss_total += loss.item()
            epoch_loss += loss.item()

            if print_every and (i+1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('\tCurrent Loss: %.4f' % print_loss_avg)

    return epoch_loss / len(data_loader)

In [22]:
def summary(
    model,
    sample, 
    idx2token=None
    ):
    model.predict = True
    model.eval()

    # shape = [seq_len, 1]
    input_batch = sample['src']
    # list
    input_len = sample['src_len']

    output_tokens = model(input_batch, input_len)
    output_tokens = [idx2token[t] for t in output_tokens]

    return " ".join(output_tokens)


In [23]:
INPUT_DIM = len(Article.vocab.stoi)
OUTPUT_DIM = len(Article.vocab.stoi)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LEARNING_RATE = 1e-4
N_EPOCHS = 200
CLIP = 1

bidirectional = True
gcn = GCN(INPUT_DIM, ENC_EMB_DIM, HID_DIM,ENC_EMB_DIM,0.5, device).to(device)
gat = GAT(ENC_EMB_DIM, HID_DIM,ENC_EMB_DIM,0.5, 0.05, 4, device).to(device)
enc = Encoder(gcn, INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, device, ENC_DROPOUT, bidirectional)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, bidirectional)
model = Seq2Seq(enc, dec, device, basic_dict=Article.vocab.stoi).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [24]:
best_valid_loss = float('inf')
count = 0

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss = train(model, train_iter, optimizer, CLIP)
    valid_loss = evaluate(model, val_iter)
    end_time = time.time()
    if count == 5:
        break
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'gcn+seq2seq(node).pt')
    else:
        count += 1

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        
    print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

  0%|                                                                                         | 0/6250 [00:00<?, ?it/s]D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarni

	Current Loss: 7.6234


 16%|███████████▊                                                              | 1000/6250 [1:41:57<9:04:00,  6.22s/it]

	Current Loss: 6.9809


 24%|█████████████████▊                                                        | 1500/6250 [2:32:58<9:33:24,  7.24s/it]

...

	Current Loss: 2.9144


 32%|███████████████████████▋                                                  | 2000/6250 [1:54:01<4:07:52,  3.50s/it]

	Current Loss: 2.9429


 39%|████████████████████████████▋                                             | 2422/6250 [2:18:11<3:38:24,  3.42s/it]


KeyboardInterrupt: 

In [25]:
from rouge import Rouge

model.load_state_dict(torch.load('gcn+seq2seq(node).pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [37]:
dct = Article.vocab.stoi
r1 = 0
r2 = 0
rl = 0
rouge = Rouge()
for i in range(10000):
    sample = test[i].article
    label = ' '.join(test[i].title)
    sample = [dct['<bos>']] + [dct[ele] for ele in sample] + [dct['<eos>']]
    test_sample = {}
    test_sample["src"] = torch.tensor(sample, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(sample)]
    predict = ''.join(summary(model, test_sample, Article.vocab.itos))[6:-6]
    score = rouge.get_scores(label, predict)[0]
    r1 += score['rouge-1']['r']
    r2 += score['rouge-2']['r']
    rl += score['rouge-l']['r']
    print(label)
    print(predict)
    print(r1/(i+1),r2/(i+1),rl/(i+1))
    print(i)
    
print(r1/10000)
print(r2/10000)
print(rl/10000)

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


boeing to cooperate with airship builder cargolifter
fed expands china < > on <
0.0 0.0 0.0
0
us says palestinian pm must control security forces
abbas calls rules to remove its stability in indonesia
0.0 0.0 0.0
1
angola zambia urged to overcome crisis
s. africa <oov> regional development in <oov> countries
0.0 0.0 0.0
2
britain steps up warnings of indonesian terror threat
oecd hails first office
0.0 0.0 0.0
3
asia foundation cooperates with yunnan in offering legal aid service
<oov> china < < unk <oov> committee for < unk > < unk > law
0.0 0.0 0.0
4
upton 's torrid offensive pace not slowing
griffey makes < < unk unk > as a < > < unk > contributed
0.0 0.0 0.0
5
as afghan leader visits u.s. villagers press claims of wrongful u.s. attack
un official leader defends first in of # nd # # years
0.023809523809523808 0.0 0.028571428571428574
6
parents in prayer death get probation some jail
man executes man in to # years after
0.03645833333333333 0.0 0.04285714285714286
7
karadzic asks to s

wartime us president picks up his peace prize
clinton 's < president on the unk > of china unk > contributed reporting
0.06529410241695635 0.004090909090909091 0.07311133311133312
54
iraq tries desperately to prove it is disarming in advance of crucial u.n. report
iraq 's unk > takes <
0.06710432677855832 0.004017857142857143 0.07478196406767837
55
drc delegation asks tanzania to mediate dialogue
president hails s. africa of president president
0.06592705788770642 0.003947368421052632 0.07346999978578928
56
man ordered to stay away from tom cruise
<oov> 's # s first of of his time of the time
0.06479038447584941 0.003879310344827586 0.07220327565155153
57
assailants kill # # in algeria reports say
top-seeded algerian republic dies # in hospital in indonesia 's largest # # # #
0.06708207287456383 0.006234866828087168 0.07267440657271167
58
giuliani revamps child welfare system
the s <oov> <oov> in the city city of a year
0.06596403832665444 0.006130952380952381 0.07146316646316647
59
wa

with giants opener a week away bonds is on the witness stand
<oov> of < a
0.09822632951648959 0.01976524523694335 0.10112654640956525
105
u.s. troops arrive in kuwait for joint exercises
us navy seizes < of first # , # # # # # dollars from kuwait
0.09789243858642893 0.019580523318841078 0.10103105617124307
106
barcelona names cruyff honorary president
albert albert champion retires # # at world # # # # #
0.09698602711803607 0.019399222176999957 0.10009558342891675
107
german shares close mixed
ecb stock exchange ends
0.096096247052733 0.019221247661614635 0.0991772753240643
108
irish stocks down six percent
stocks rise lower on economic growth of asian bank
0.09623274581690006 0.019046509046509046 0.09928576474031017
109
ross meets netanyahu in bid to revive < unk > process
arafat chief 's israeli state policy of china unk > contributed reporting for < article
0.09716758594467573 0.01951841951841952 0.09959249959249956
110
amputee athletes gird for world competitions
harvard world pres

floods kill nine as storms hit algeria
mysterious disease reported of southern central korea
0.09485058563484974 0.014562870028457074 0.09797759220836141
155
wiesenthal center asking check of eva peron link to nazis
world group of <oov> . in
0.09552032712762139 0.014470112894517858 0.09862741646181133
156
indian government distances itself from celebrity guru
pak unk > 's first
0.0949157680951681 0.01437852990151458 0.09800319230698974
157
palestinians stage demonstrations in territories
army of < of major bank in israel city
0.09501762559841302 0.014288098895844677 0.09817298355034201
158
thousands stranded on rooftops as heavy rains cause floods
floods damages becomes more more
0.09567376543842293 0.014198798277745648 0.09912190240315237
159
us to increase tsunami aid to # # # million dollars
uae announces emergency to for dprk to combat # years
0.09694287248538926 0.014110606984095053 0.09988650066289814
160
tour of flanders cycling results
swiss wins super-g of nordic # # # #
0.097

...

pre-historic animal fossils found in north china
rare ancient china of
0.09390116331653993 0.010139432059714393 0.10015125720569978
9772
u.s. may protect prairie dog
southern develops company to unk > of <
0.0938915560765853 0.01013839467153558 0.10014101050453283
9773
broadway stock is in a free fall
< company 's <oov> <oov> <oov> securities bonds in late #
0.09389125096505921 0.010137357495610105 0.10014213276546445
9774
nuggets backup eduardo najera a star in mexico
north korea national < in national history of history history
0.09389187583709634 0.010136320531872828 0.10014650214047383
9775
strong quake shakes eastern aegean sea
moderate tremor quakes in <oov> of no
0.09388227249498352 0.010135283780258644 0.10013625906978338
9776
extortion case filed against former bangladeshi pm sheikh hasina
fiji pm defends # <oov> . #
0.09388971618430358 0.01013424724070247 0.10014647217480796
9777
bush to warn of energy crisis
president president <oov> in asia 's economy
0.0938801150271112 0.0

iran refuses immediate tougher nuclear probes but agrees to more iaea talks
u.n. agency <oov> china to < unk > on u.s. military issues
0.09390610026877035 0.010219101085121573 0.1002610882299437
9822
new zealand overwhelms ireland # # - # # in rugby test
first zealand new leader loses second time of
0.09392198930579511 0.010218060867177239 0.10026360644164667
9823
northeast china 's harbin airport gets new name
< < < > < >
0.09391242981578943 0.010217020860982107 0.10025340149442614
9824
a stylish facelift with a few wrinkles
dprk s > unk unk unk > in # st china province
0.09390287227153787 0.010215981066471525 0.10024319862433716
9825
chinese president expresses sympathy with bolivian landslide victims
vice president praises chinese guests in # # years
0.09391593010708796 0.01021494148358087 0.10024571788773144
9826
khartoum southern sudanese rebels agree to continue peace talks
dprk <oov> cease-fire holds <oov> 's of military power in the of of < unk > < unk > < unk > < > contributed

philippines adopts < unk > housing scheme
indonesian currency currency sector improves to combat control of indonesian article
0.09379728289758402 0.010242328343657145 0.10013026783441725
9872
hong kong stocks open higher dec. #
hong kong index closes higher
0.09384854912374388 0.010266610060454425 0.10018089268069694
9873
britain freezes assets of zimbabwe 's ruling party
philip publishes first of <oov> in indonesia
0.09385351201931182 0.010265570403739443 0.10018521432628442
9874
tibetan activists to step up efforts for detained scholar
burma denies accusation in in # china
0.09384400882854436 0.010264530957566523 0.10017507001539679
9875
afghan leader driven from kabul makes stand in north
< military unk war
0.09383450756208404 0.010263491721871722 0.1001649277586371
9876
canadian firm resumes diamond mining in sierra leone
marine national <oov> military < unk > military in south
0.09383513172612919 0.010262452696591111 0.10016603589625124
9877
china devises new rules to invigorate 

leeds finance director quits
chief 's <oov> resigns at <oov> .
0.0937876890696154 0.010243020459148182 0.10011818971384252
9921
congress takes another swing at baseball anti-monopoly exemption
us senate takes rules rules rules in
0.09379263409166393 0.010241988208774389 0.10012825540065964
9922
jordanian premier meets with turkish ambassador
lebanon prime prime minister <oov> china 's democracy
0.0937831829999578 0.010240956166431708 0.10011816589487561
9923
pakistan taliban chief mehsud dead
pakistan says it 's is dead from the to > military forces says says no injuries reported in # years
0.09378380937950441 0.010239924332057256 0.10011927349640873
9924
special envoy of president clinton meets general abacha
senior official official <oov> <oov> in china
0.09377436108115869 0.010238892705588178 0.10010918692845625
9925
worldwide server workstation shipments rise in # rd quarter
supercomputer supercomputer electronics sales
0.09376491468636862 0.010237861286961646 0.10009910239265203
9

macao follows hk 's move to provide deposits guarantee to banks
hksar issues chief defends executive 's # . # # yuan yuan in china
0.09375358085001943 0.01020987639826465 0.10010706244214088
9970
world bank finances education project in sri lanka
china 's <oov> development bank of <oov> development
0.09375671426549777 0.010208852543832412 0.10011373709158176
9971
chinese vice-premier hails achievements in building qinghai-tibet railway
china <oov> underscores <oov> china of control in tibet
0.09375845440355508 0.010207828894725442 0.10011802300412218
9972
mccain hammers dishonest obama on national security
mccain acknowledges concedes in his than
0.093765764230331 0.010206805450881975 0.10012469521623993
9973
most monks in cyberspace are traditional
china unk < > women in
0.0937730725914775 0.010205782212240283 0.1001313660905708
9974
tropical storm karen makes landfall in nova scotia expected to
drought tropical hits <oov> of the north of china s < of the unk > < unk > contributed
0.0

In [35]:
dct = Article.vocab.stoi
sample = test[500].article
print(sample)
print(test[500].title)
sample = [dct['<bos>']] + [dct[ele] for ele in sample] + [dct['<eos>']]

test_sample = {}
test_sample["src"] = torch.tensor(sample, dtype=torch.long, device=device).reshape(-1, 1)
test_sample["src_len"] = [len(sample)]

print(summary(model, test_sample, Article.vocab.itos), end="\n\n")

['the', 'pakistani', 'government', 'saturday', 'handed', 'over', '#', '#', 'more', 'buses', 'to', 'the', 'afghan', 'government', 'as', 'part', 'of', 'its', 'contribution', 'to', 'the', 'reconstruction', 'in', 'afghanistan', ',', 'according', 'to', 'the', 'associated', 'press', 'of', 'pakistan', '.']
['pakistan', 'hands', 'over', '#', '#', 'more', 'buses', 'to', 'afghanistan']


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


<bos> military military <oov> operation from asia in india <eos>



In [ ]:
dct = Article.vocab.stoi
r1 = 0
r2 = 0
rl = 0
rouge = Rouge()
for i in range(10000):
    sample = test[i].article
    label = ' '.join(test[i].title)
    sample = [dct['<bos>']] + [dct[ele] for ele in sample] + [dct['<eos>']]
    test_sample = {}
    test_sample["src"] = torch.tensor(sample, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(sample)]
    predict = ' '.join(summary(model, test_sample, Article.vocab.itos))
    score = rouge.get_scores(label, predict)[0]
    r1 += score['rouge-1']['r']
    r2 += score['rouge-2']['r']
    rl += score['rouge-l']['r']
#     print('--------------------------')
    print(r1/(i+1),r2/(i+1),rl/(i+1))
    print(i)
    
print(r1/10000)
print(r2/10000)
print(rl/10000)

In [39]:
total = pd.read_csv('total.csv')

In [48]:
print('Article:',total.iloc[10,:]['article'])
print('Title:',total.iloc[10,:]['title'])

Article: new zealand share prices closed #.## percent higher monday in subdued trading ahead of a us holiday , dealers said .
Title: new zealand stocks close #.## percent higher


Article: australia 's current account deficit shrunk by a record #.## billion dollars -lrb- #.## billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed .